In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

In [2]:
# Clona apenas o diretório de imagens usando sparse checkout
!git clone --depth 1 --filter=blob:none --sparse https://github.com/MRW-Code/cmac_particle_flow.git
%cd cmac_particle_flow
!git sparse-checkout set images
%cd images

# Verifica o conteúdo
!ls

Cloning into 'cmac_particle_flow'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 22 (delta 3), reused 11 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (22/22), 7.51 KiB | 7.51 MiB/s, done.
Resolving deltas: 100% (3/3), done.
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 1), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (8/8), 5.75 KiB | 5.75 MiB/s, done.
Resolving deltas: 100% (1/1), done.
/content/cmac_particle_flow
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 98 (delta 0), reused 98 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (98/98), 349.55 MiB | 36.23 MiB/s, done.
Updating files: 100% (174/174), done.
/content/cmac_particle_flow

In [10]:
from PIL import Image
from skimage.feature import local_binary_pattern
from skimage import io, color
import os
import pandas as pd
import numpy as np

# Diretório base contendo as pastas com as imagens
base_path = '/content/cmac_particle_flow/images'
folders = ['Cohesive', 'Easyflowing', 'Freeflowing']

# Divisões desejadas
split_list = [2, 3, 4, 5, 6, 7, 8, 9, 10]

# Tamanho final de cada subimagem
resize_shape = (384, 384)

# Loop principal para todas as divisões
for split in split_list:
    features_all = []
    for folder in folders:
        folder_path = os.path.join(base_path, folder)
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
                img_path = os.path.join(folder_path, filename)
                img = Image.open(img_path).convert('L')
                w, h = img.size

                tile_w = w // split
                tile_h = h // split

                for i in range(split):
                    for j in range(split):
                        left = j * tile_w
                        upper = i * tile_h
                        right = (j + 1) * tile_w
                        bottom = (i + 1) * tile_h
                        tile = img.crop((left, upper, right, bottom))
                        tile_resized = tile.resize(resize_shape)
                        tile_np = np.array(tile_resized)
                        # Aplicando o LBP
                        radius = 1
                        n_points = 8 * radius
                        METHOD = 'uniform'
                        lbp_img = local_binary_pattern(tile_np, P=n_points, R=radius, method=METHOD)
                        n_bins = int(lbp_img.max() + 1)
                        hist, _ = np.histogram(lbp_img.ravel(), bins=n_bins, range=(0, n_bins), density=True)
                        # Armazena a linha completa com identificadores
                        features_all.append(
                            [filename, folder] + list(hist)
                        )

    # Criar DataFrame
    colunas = ['ImageId', 'Classe'] + [f'Feature_LBP_{i+1}' for i in range(len(hist))]
    df = pd.DataFrame(features_all, columns=colunas)

    # Salvar CSV
    csv_name = f'features_lbp_individual_split_{split}x{split}.csv'
    df.to_csv(csv_name, index=False)
    print(f'Salvo: {csv_name} com shape {df.shape}')

Salvo: features_lbp_individual_split_2x2.csv com shape (388, 12)
Salvo: features_lbp_individual_split_3x3.csv com shape (873, 12)
Salvo: features_lbp_individual_split_4x4.csv com shape (1552, 12)
Salvo: features_lbp_individual_split_5x5.csv com shape (2425, 12)
Salvo: features_lbp_individual_split_6x6.csv com shape (3492, 12)
Salvo: features_lbp_individual_split_7x7.csv com shape (4753, 12)
Salvo: features_lbp_individual_split_8x8.csv com shape (6208, 12)
Salvo: features_lbp_individual_split_9x9.csv com shape (7857, 12)
Salvo: features_lbp_individual_split_10x10.csv com shape (9700, 12)
